In [ ]:

# my helper functions for this project
import importlib
import helper



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 


# set plot font size for all plots
plt.rcParams.update({'font.size': 16})  # applies to everything

np.random.seed(1234)

# positive class: 1 
# negative class: 0
# high score -> more likely to be positive

# number of examples in each class
nN = 4*2000
nP = 2000
n = nN + nP

meanP=0.6
meanN=0.4
scalePN=0.12

# scores for each class generated from two normal distributions
scoresN=np.random.normal(loc=meanN, scale=scalePN, size=nN)
scoresP=np.random.normal(loc=meanP, scale=scalePN, size=nP)

# plot the scores - one histogram for each class
def plotScoreDistributions(scores, labels):
    plt.hist(scores[labels==0], density=False, bins=30, alpha=0.5, label="Negative class", color="skyblue")
    plt.hist(scores[labels==1], density=False, bins=30, alpha=0.5, label="Positive class", color="green")
    plt.legend(loc='upper right')
    plt.xlabel('Score')
    plt.ylabel('Number of examples')


# create labels for the two classes
labelsN=np.zeros(nN)
labelsP=np.ones(nP)

# create data frame of scores and labels
scores= np.concatenate((scoresN, scoresP))
labels= np.concatenate((labelsN, labelsP))

d = {'score': scores, 'label':labels}

df = pd.DataFrame(data=d)

df = df.set_index("score")
df = df.sort_values('score')
df.reset_index(inplace=True)

# checking
df.size
#print(df.head(n=10))
#print(df.tail(n=10))
print(min(df.score))
print(max(df.score))

print('class distribution:', nP/(nN+nP))

plotScoreDistributions(df.score, df.label)

# fix scores outside 0 and 1
print('Min:', min(df.score), " Max: ", max(df.score))

df.loc[df['score'] < 0, 'score'] = 0
df.loc[df['score'] > 1, 'score'] = 1
print('Min:', min(df.score), " Max: ", max(df.score))


In [ ]:
# calculate AUC

from sklearn import metrics
#xx = df.dropna(subset="label")
#auc = metrics.roc_auc_score(xx['label'], xx['score'])

auc = metrics.roc_auc_score(df['label'], df['score'])
print("AUC:", auc)

In [ ]:
# calibrate scores
from scipy.stats import norm

def myCalibrate(score):
    piP = nP/(nN+nP)

    scoresCal = piP*norm.pdf(score, meanP, scalePN)/((1-piP)*norm.pdf(score, meanN, scalePN)+piP*norm.pdf(score, meanP, scalePN))
    plt.scatter(score, scoresCal)
    plt.xlabel('Original score')
    plt.ylabel('Calibrated score')
    plt.axline((0, 0), slope=1, color='red')


    return(scoresCal)

df['scoreCal'] = myCalibrate(df.score)

df


In [ ]:
# plot the calibrated score distributions

plotScoreDistributions(df.scoreCal, df.label)


In [ ]:
# calculate TPR and FPR
importlib.reload(helper)
df = helper.calculateROCPoints(df)

print(df.head(n=10))
print(df.tail(n=10))

In [ ]:
# Plot ROC curve.
#
# When plotLossIsometrics=True loss isometrics are also shown along with example uncalibrated and calibrated
# scores along the ROC curve, to show the difference in loss due to calibration error. 


def plotROCCurve(plotLossIsometrics=False, showExampleScores=False):
    # plot roc curve
    ax = df.plot(kind = 'line', x = 'fpr', y ='tpr', legend=False, linewidth=2, color='skyblue')

    if (plotLossIsometrics==True):
        # isometrics for cost proportion 0.8
        for loss in np.arange(0.01,2.01, 0.1):
            c=0.8
            piP = nP/(nN+nP)
            piN = 1 - piP
            gradient = (c/(1-c))*piN/piP
            intercept = 1 - (loss/(2*piP*(1-c)))

            plt.axline((0, intercept), slope=gradient, color='pink', linestyle='--')

    if (showExampleScores==True):
        # print score at intervals along the roc curve to see how the original vs calibrated scores differ
        for i in [100, 1000, 5000, 8000, 9000, 9800]:
            plt.text(df.loc[i, 'fpr'], df.loc[i, 'tpr'] , "{:.2f}". format(df.loc[i, 'score']) , fontsize=10, backgroundcolor='white', alpha=0.5, bbox=dict(boxstyle='square', fc='white', ec='none'))
            plt.text(df.loc[i, 'fpr'], df.loc[i, 'tpr']-0.05 , "{:.2f}". format(df.loc[i, 'scoreCal']) , fontsize=10, backgroundcolor='white', color='red', alpha=0.5, bbox=dict(boxstyle='square', fc='white', ec='none'))
        
    ax.set_ylabel("TPR")
    ax.set_xlabel("FPR")

    ax.set_xlim([0,1])
    ax.set_ylim([0,1])


plotROCCurve()
plotROCCurve(False, True)
plotROCCurve(True, True)


In [ ]:
###
### plotting the cost curve

import matplotlib.pyplot as plt 

# Calculate loss for different costs
piN = nN/(nN+nP)
piP = 1 - piN

b = 2
c_0 = 1
c = c_0 / 2

fig, ax = plt.subplots()
    
# plot line in cost space for each F0 F1 pair in ROC space
for i in range(1,df.shape[0]):

    gradient = 2*(piN*df.loc[i,'fpr'] - piP*(1-df.loc[i,'tpr']))
    intercept = 2*piP * (1-df.loc[i,'tpr'])
    ax.axline((0, intercept), slope=gradient, color='C0')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)




In [ ]:
# Cost curve without plotting all the lines

importlib.reload(helper)
dfLossCost = helper.lowerEnvelope(df)

ax = dfLossCost.plot(kind = 'line', x = 'cost', y ='loss', legend=False, color='skyblue', ylim=[0,1])
ax.set_ylabel("Loss")
ax.set_xlabel("Cost")



In [ ]:
# calculate brier loss for the original score and the calibrated scores

df['brier_loss'] = 2*(1-df['score'])*piP*(1-df['tpr']) + 2*df['score']*piN*df['fpr']
df['brier_loss_cal'] = 2*(1-df['scoreCal'])*piP*(1-df['tpr']) + 2*df['scoreCal']*piN*df['fpr']



In [ ]:
# plot cost curve and brier curve (of uncalibrated scores)
ax = dfLossCost.plot(kind = 'line', x = 'cost', y ='loss', color='skyblue', ylim=[0,1], linestyle=':', linewidth=3, label='Model lower envelope cost curve')
df.plot(kind = 'line', x = 'score', y ='brier_loss', ax=ax, color='C2', linestyle='-', linewidth=2, label='Model Brier curve')
ax.set_ylabel("Loss")
ax.set_xlabel("Cost proportion")
df


In [ ]:

# same plot but checking that the calibrated scores version is the same as the cost curve

ax = dfLossCost.plot(kind = 'line', x = 'cost', y ='loss', legend=False, color='skyblue', ylim=[0,1])
df.plot(kind = 'line', x = 'score', y ='brier_loss', ax=ax, color='C7', linestyle='-', label='Model Brier curve')
df.plot(kind = 'line', x = 'scoreCal', y ='brier_loss_cal', ax=ax, color='C8', linestyle='--', label='Model Brier curve cal')
ax.set_ylabel("Loss")
ax.set_xlabel("Cost proportion")

ax.set_xlim([0,1])


In [ ]:
# calculate net benefit for DCA

#df['netben'] = df.tpr*nP/n - (df.fpr*nN/n)*(df.score/(1-df.score))
df['netben'] = df['tpr']*nP/n - (df['fpr']*nN/n)*(df['score']/(1-df['score']))
df['netben_treatall'] = nP/n - (nN/n)*(df['score']/(1-df['score']))

df

In [ ]:
# plot DCA curve

fig, ax = plt.subplots()
ax.set_xlim(0, 1)
ax.set_ylim(-1, 1)
ax.axline((0, 0), slope=0, color="#FBB6B6", markersize=1, linestyle='-', linewidth=2, label="Treat none")
df.plot(kind = 'line', x = 'score', y ='netben', ax=ax, color='C2', linewidth=2, linestyle='-', label='Model decision curve')
df.plot(kind = 'line', x = 'score', y ='netben_treatall', ax=ax, color='C2', linestyle='--', linewidth=2, label='Treat all')

ax.set_xlabel('Threshold score')
ax.set_ylabel('Net benefit')

ax.legend();


In [ ]:
# standard decision curve

df['netben_cal'] = df['tpr']*nP/n - (df['fpr']*nN/n)*(df['scoreCal']/(1-df['scoreCal']))

importlib.reload(helper)
lowerEnv = helper.lowerEnvelopeNB(df)
plt.plot(lowerEnv['cost'], lowerEnv['nb'], color='skyblue', linestyle=':', linewidth=3, label='Upper envelope decision curve')

plt.plot(df['score'], df['netben'], color='C2', linestyle='-', linewidth=2, label='Model decision curve')
plt.plot(df['score'], df['netben_treatall'], color='C2', linewidth=2, linestyle='--', label='Treat all')
#plt.plot(df['scoreCal'], df['netben_cal'], color='yellow', linestyle='--', label='Model, calibrated probabilities')

plt.axline((0, 0), (1, 0), color='red', markersize=1, linestyle='-', linewidth=1, label="Treat none")

plt.xlabel('Decision threshold')
plt.ylabel('Net benefit')

plt.xlim(0,1)
plt.ylim(-1,1)
#lowerEnv

plt.legend()


In [ ]:
###
### plot cost curves including the treat all and treat none lines


df['brier_loss_treatall'] = 2*(1-df['score'])*piP*(1-1) + 2*df['score']*piN*1
df['brier_loss_treatnone'] = 2*(1-df['score'])*piP*(1-0) + 2*df['score']*piN*0

##
## cost curve with cost proportion as the operating condition

# plot cost curve and brier curve (of uncalibrated scores)
ax = dfLossCost.plot(kind = 'line', x = 'cost', y ='loss', color='skyblue', linestyle=':', linewidth=3, ylim=[0,1], label='Lower envelope cost curve')
df.plot(kind = 'line', x = 'score', y ='brier_loss', ax=ax, color='C2', linestyle='-', linewidth=2, label='Model Brier curve')
df.plot(kind = 'line', x = 'score', y ='brier_loss_treatall', ax=ax, color='C2', linestyle='--', label='Predict all as positive')
df.plot(kind = 'line', x = 'score', y ='brier_loss_treatnone', ax=ax, color='red', linestyle='-', label='Predict all as negative', linewidth=1)

ax.set_ylabel("Loss")
ax.set_xlabel("Cost proportion")
ax.set_xlim([0,1])


##
## cost curve with skew as the operating condition


df['brier_loss_skew'] = (1-df['score'])*(1-df['tpr']) + df['score']*df['fpr']
df['brier_loss_treatall_skew'] = (1-df['score'])*(1-1) + df['score']*1
df['brier_loss_treatnone_skew'] = (1-df['score'])*(1-0) + df['score']*0
#df['skew'] = 2*(1-df['score'])*piP*(1-1) + 2*df['score']*piN*1

dfLossSkew = helper.lowerEnvelopeSkew(df)


# plot cost curve and brier curve (of uncalibrated scores)
ax = dfLossSkew.plot(kind = 'line', x = 'skew', y ='loss', color='blue', linestyle=':', ylim=[0,1], label='Model cost curve')
df.plot(kind = 'line', x = 'score', y ='brier_loss_skew', ax=ax, color='green', linestyle='-', label='Model Brier curve')
df.plot(kind = 'line', x = 'score', y ='brier_loss_treatall_skew', ax=ax, color='green', linestyle='--', label='Predict all as positive')
df.plot(kind = 'line', x = 'score', y ='brier_loss_treatnone_skew', ax=ax, color='red', linestyle='-', label='Predict all as negative', linewidth=1)

ax.set_ylabel("Loss")
ax.set_xlabel("Skew")

ax.set_xlim([0,1])
df



In [ ]:
# General decision curve function that shows it
# - with or without Brier loss isometrics
# - using DCA or Brier costs

def decisionCurve(df, showIsometrics=False, nbBrierCosts=False):

    importlib.reload(helper)
    lowerEnv = helper.lowerEnvelopeNB(df, brierCosts=nbBrierCosts)
    plt.plot(lowerEnv['cost'], lowerEnv['nb'], color='skyblue', linestyle=':', linewidth=3, label='Upper envelope decision curve')

    if (nbBrierCosts==False):
        plt.plot(df['score'], df['netben'], color='C2', linestyle='-', linewidth=2, label='Model decision curve')
        plt.plot(df['score'], df['netben_treatall'], color='C2', linestyle='--', linewidth=2, label='Treat all')
    else:
        plt.plot(df['score'], df['netben_briercosts'], color='C2', linestyle='-', linewidth=2, label='Model decision curve (Brier costs)')
        plt.plot(df['score'], df['netben_treatall_briercosts'], color='C2', linewidth=2, linestyle='--', label='Treat all (Brier costs)')


    plt.axline((0, 0), (1, 0), color='red', markersize=1, linestyle='-', linewidth=1, label="Treat none")

  
    if (showIsometrics==True):
        for loss in np.arange(0,1.5, 0.1):
                
                nb_costs = []

                for c in np.arange(0,1, 0.01):
                    
                    # nb from loss with standard costs for each
                    if (nbBrierCosts==False):
                        nb = piP - loss/(2*(1-c))
                    else:
                        # nb from loss with Brier costs for each
                        nb = 2*piP*(1-c) - loss

                    nb_cost = {'cost':c, 'nb':nb}
                    nb_costs.append(nb_cost)


                dfNBCost = pd.DataFrame(nb_costs)
                plt.plot(dfNBCost['cost'], dfNBCost['nb'], color='pink', linestyle=':')

                textXPos = 0.7+(1-loss)/5
                if (nbBrierCosts==False):
                    textYPos = (piP - loss/(2*(1-textXPos)))
                else:
                    textYPos = 2*piP*(1-textXPos) - loss
                    
                if (textYPos >=-1):
                    plt.text(textXPos,textYPos , "{:.2f}". format(loss) , fontsize=8, backgroundcolor='white', alpha=0.5, bbox=dict(boxstyle='square', fc='white', ec='none'))


    plt.xlabel('Threshold score')
    plt.ylabel('Net benefit');

    plt.xlim(0,1)
    plt.ylim(-1,1)

    plt.legend(fontsize=12)
    plt.show()
    
decisionCurve(df, showIsometrics=False)
decisionCurve(df, showIsometrics=False, nbBrierCosts=True)
decisionCurve(df, showIsometrics=True)
decisionCurve(df, showIsometrics=True, nbBrierCosts=True)


In [ ]:
### Cost curve with DCA costs
### 


df['DCA_loss_treatall'] = (df['score']/(1-df['score']))*piN
df['DCA_loss_treatnone'] = piP
df['DCA_loss'] = piP*(1-df['tpr']) + (df['score']/(1-df['score']))*piN*df['fpr']

importlib.reload(helper)
dfDCALossCost = helper.lowerEnvelopeDCALoss(df)


# plot cost curve and brier curve (of uncalibrated scores)
ax = dfDCALossCost.plot(kind = 'line', x = 'cost', y ='loss', color='skyblue', linestyle=':', linewidth=2, ylim=[0,1], label='Lower envelope cost curve')
df.plot(kind = 'line', x = 'score', y ='DCA_loss', ax=ax, color='C2', linestyle='-', label='Brier curve with DCA loss')
df.plot(kind = 'line', x = 'score', y ='DCA_loss_treatall', ax=ax, color='C2', linestyle='--', label='Predict all as positive')
df.plot(kind = 'line', x = 'score', y ='DCA_loss_treatnone', ax=ax, color='red', linestyle='-', label='Predict all as negative', linewidth=1)

ax.set_ylabel("Loss")
ax.set_xlabel("Cost proportion")
ax.set_xlim([0,1])


In [ ]:

## reverse DCA, in terms of interventions avoided

df['netben_interventions_avoided'] = piN*(1-df['fpr']) - (piP*(1-df['tpr'])*(1-df['score'])/df['score'])
df['netben_interventions_avoided_treat_all'] = piN - (piP*(1-df['score'])/df['score'])
df['netben_interventions_avoided_treat_none'] = 0


plt.plot(df['score'], df['netben_interventions_avoided'], color='C2', linestyle='-', label='Decision curve')
plt.plot(df['score'], df['netben_interventions_avoided_treat_all'], color='C2', linestyle='--', label='Treat all')
plt.plot(df['score'], df['netben_interventions_avoided_treat_none'], color='red', linestyle='-', label='Treat none')


plt.xlim(0,1)
plt.ylim(-1, 1)

plt.xlabel('Decision threshold')
plt.ylabel('Net benefit for negative class');

plt.legend()


In [ ]:

# class specific cost curves


# loss weighted by class distribution, showing contributions of each class to Brier curve across thresholds

df['brier_loss_pos'] = 2*(1-df['score'])*piP*(1-df['tpr']) 
df['brier_loss_neg'] = 2*df['score']*piN*df['fpr']


# plot cost curve and brier curve (of uncalibrated scores)
ax = dfLossCost.plot(kind = 'line', x = 'cost', y ='loss', color='skyblue', linestyle=':', linewidth=3, ylim=[0,1], label='Lower envelope cost curve')
df.plot(kind = 'line', x = 'score', y ='brier_loss', ax=ax, color='C2', linestyle='-', label='Brier curve')
df.plot(kind = 'line', x = 'score', y ='brier_loss_treatall', ax=ax, color='C2', linestyle='--', label='Predict all as positive')
df.plot(kind = 'line', x = 'score', y ='brier_loss_treatnone', ax=ax, color='red', linestyle='-', label='Predict all as negative', linewidth=1)

df.plot(kind = 'line', x = 'score', y ='brier_loss_pos', ax=ax, color='C2', linestyle=':', label='Brier curve, positives')
df.plot(kind = 'line', x = 'score', y ='brier_loss_neg', ax=ax, color='C2', linestyle='-.', label='Brier curve negatives')

ax.set_ylabel("Loss")
ax.set_xlabel("Cost proportion")
ax.set_xlim([0,1])

plt.legend(fontsize=12, loc='upper right')

# loss, not weighted by class distribution (i.e. as if test dataset only includes one class)

df['brier_loss_pos'] = 2*(1-df['score'])*(1-df['tpr']) 
df['brier_loss_neg'] = 2*df['score']*df['fpr']


# plot cost curve and brier curve (of uncalibrated scores)
ax = dfLossCost.plot(kind = 'line', x = 'cost', y ='loss', color='skyblue', linestyle=':', linewidth=3, ylim=[0,1], label='Lower envelope cost curve')
df.plot(kind = 'line', x = 'score', y ='brier_loss', ax=ax, color='C2', linestyle='-', label='Brier curve')
df.plot(kind = 'line', x = 'score', y ='brier_loss_treatall', ax=ax, color='C2', linestyle='--', label='Predict all as positive')
df.plot(kind = 'line', x = 'score', y ='brier_loss_treatnone', ax=ax, color='red', linestyle='-', label='Predict all as negative', linewidth=1)



df.plot(kind = 'line', x = 'score', y ='brier_loss_pos', ax=ax, color='C2', linestyle=':', label='Brier curve, positives')
df.plot(kind = 'line', x = 'score', y ='brier_loss_neg', ax=ax, color='C2', linestyle='-.', label='Brier curve negatives')



ax.set_ylabel("Loss")
ax.set_xlabel("Cost proportion")
ax.set_xlim([0,1])

plt.legend(fontsize=12)


In [ ]:

# Cost curves with NB isometrics

df['brier_loss_treatall'] = 2*(1-df['score'])*piP*(1-1) + 2*df['score']*piN*1
df['brier_loss_treatnone'] = 2*(1-df['score'])*piP*(1-0) + 2*df['score']*piN*0

##
## cost curve with cost proportion as the operating condition

# plot cost curve and brier curve (of uncalibrated scores)
plt.plot(dfLossCost['cost'], dfLossCost['loss'], color='skyblue', linestyle=":", label='Lower envelope cost curve')
plt.plot(df['score'], df['brier_loss'], color='C2', linestyle='-', label='Model Brier curve')
plt.plot(df['score'], df['brier_loss_treatall'], color='C2', linestyle='--', label='Predict all as positive')
plt.plot(df['score'], df['brier_loss_treatnone'], color='red', linestyle='-', label='Predict all as negative', linewidth=1)


def lossIsometrics(brierCosts=False):
    for nb in np.arange(-100,0.2, 0.1):
            
            c_losses = []

            for c in np.arange(0,1, 0.01):

                #nb = piP - loss/(2*(1-c))
                if (brierCosts==False):
                    loss = (piP - nb)*2*(1-c)
                else:
                    loss = 2*piP*(1-c) - 2*nb

                c_loss = {'cost':c, 'loss':loss}
                c_losses.append(c_loss)


            dfCLoss = pd.DataFrame(c_losses)
            
            plt.plot(dfCLoss['cost'], dfCLoss['loss'], color='pink', linestyle=':')

            textXPos = 0.4

            if (brierCosts==False):
                textYPos = (piP - nb)*2*(1-textXPos)
            else:
                textYPos = 2*piP*(1-textXPos) - 2*nb
            
            if (textYPos <1):
                #nPer100 = nb*100
                plt.text(textXPos,textYPos , "{:.2f}". format(nb) , fontsize=8, backgroundcolor='white', alpha=0.5, bbox=dict(boxstyle='square', fc='white', ec='none'))


lossIsometrics(True)

plt.ylabel("Loss")
plt.xlabel("Cost proportion")
plt.xlim([0,1])
plt.ylim([0,1])

plt.legend(fontsize=12)
